Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [2]:
labels = [{'name':'A', 'id':1},{'name':'B', 'id':2},{'name':'C', 'id':3},
{'name':'D', 'id':4},{'name':'E', 'id':5},{'name':'F', 'id':6},{'name':'G', 'id':7},{'name':'H', 'id':8},
{'name':'I', 'id':9},{'name':'J', 'id':10},{'name':'K', 'id':11},{'name':'L', 'id':12},{'name':'M', 'id':13},
{'name':'N', 'id':14},{'name':'O', 'id':15},{'name':'P', 'id':16},{'name':'Q', 'id':17},{'name':'R', 'id':18},
{'name':'S', 'id':19},{'name':'T', 'id':20},{'name':'U', 'id':21},{'name':'V', 'id':22},{'name':'W', 'id':23},
{'name':'X', 'id':24},{'name':'Y', 'id':25},{'name':'Z', 'id':26}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [3]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [3]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  61% (1388/2251)
Updating files:  62% (1396/2251)
Updating files:  63% (1419/2251)
Updating files:  64% (1441/2251)
Updating files:  65% (1464/2251)
Updating files:  66% (1486/2251)
Updating files:  67% (1509/2251)
Updating files:  68% (1531/2251)
Updating files:  69% (1554/2251)
Updating files:  70% (1576/2251)
Updating files:  71% (1599/2251)
Updating files:  72% (1621/2251)
Updating files:  73% (1644/2251)
Updating files:  74% (1666/2251)
Updating files:  75% (1689/2251)
Updating files:  76% (1711/2251)
Updating files:  77% (1734/2251)
Updating files:  78% (1756/2251)
Updating files:  79% (1779/2251)
Updating files:  80% (1801/2251)
Updating files:  81% (1824/2251)
Updating files:  82% (1846/2251)
Updating files:  83% (1869/2251)
Updating files:  84% (1891/2251)
Updating files:  85% (1914/2251)
Updating files:  86% (1936/2251)
Updating files:  87% (1959/2251)
Updating files:  88% (1981/2251)
Updating files:  89% (2004/2251)
Updating files:  9

In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [4]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


# 5. Update Config For Transfer Learning

In [5]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [6]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [7]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [10]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [8]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [9]:
pipeline_config.model.ssd.num_classes = 26
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [10]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [16]:
''''import numpy as np
from tensorflow import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt

def plot_tensorflow_log(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        'compressedHistograms': 10,
        'images': 0,
        'scalars': 100,
        'histograms': 1
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    # Show all tags in the log file
    #print(event_acc.Tags())

    training_accuracies =   event_acc.Scalars('training-accuracy')
    validation_accuracies = event_acc.Scalars('validation_accuracy')

    steps = 10
    x = np.arange(steps)
    y = np.zeros([steps, 2])

    for i in xrange(steps):
        y[i, 0] = training_accuracies[i][2] # value
        y[i, 1] = validation_accuracies[i][2]

    plt.plot(x, y[:,0], label='training accuracy')
    plt.plot(x, y[:,1], label='validation accuracy')

    plt.xlabel("Steps")
    plt.ylabel("Accuracy")
    plt.title("Training Progress")
    plt.legend(loc='upper right', frameon=True)
    plt.show()


if __name__ == '__main__':
    log_file = "./logs/events.out.tfevents.1456909092.DTA16004"
    plot_tensorflow_log(log_file)''''

ImportError: cannot import name 'EventAccumulator' from 'tensorflow' (C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\__init__.py)

# 6. Train the model

In [24]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=32000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=32000


# 7. Load Train Model From Checkpoint

In [11]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [12]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-33')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [13]:
import cv2 
import numpy as np

In [14]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [16]:
FOLDER = 'D:/Machine Learning/RealTimeObjectDetection/Video_frames'
     

In [105]:
cap.release()

In [19]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [26]:
i = 0
for file in os.listdir(FOLDER):
    #print(FOLDER + file)
    frame = cv2.imread(FOLDER +'/'+ file)
    
    image_np = np.array(frame)
    i += 1
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    print(detections['detection_boxes'])
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=2,
                min_score_thresh=.1,
                agnostic_mode=False)

    #cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    cv2.imwrite('D:/Machine Learning/RealTimeObjectDetection/Detected_frames/frame' + str (i) + '.jpg', image_np_with_detections)
    '''if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break'''

[[0.         0.         1.         1.        ]
 [0.3725118  0.30474576 1.         1.        ]
 [0.         0.00401044 1.         0.9399302 ]
 [0.44170022 0.08893764 1.         1.        ]
 [0.37041235 0.1090394  1.         0.8955138 ]
 [0.00889325 0.         0.9923246  0.8488034 ]
 [0.42765927 0.         1.         0.7516179 ]
 [0.         0.11569577 1.         1.        ]
 [0.28251112 0.4532161  1.         1.        ]
 [0.         0.         1.         0.82046187]
 [0.14022368 0.         1.         0.8124503 ]
 [0.30525836 0.3750767  1.         1.        ]
 [0.         0.19774005 0.81938934 1.        ]
 [0.02851036 0.40206173 0.98164856 1.        ]
 [0.         0.         1.         0.9757477 ]
 [0.00889325 0.         0.9923246  0.8488034 ]
 [0.         0.         1.         0.8622631 ]
 [0.         0.         1.         1.        ]
 [0.         0.         1.         0.8622631 ]
 [0.02078965 0.         0.9785557  0.61072916]
 [0.00889325 0.         0.9923246  0.8488034 ]
 [0.         

In [21]:
detections = detect_fn(input_tensor)
print(detections)

{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[0.        , 0.        , 1.        , 1.        ],
        [0.00982288, 0.        , 0.99253106, 0.85240746],
        [0.        , 0.010995  , 1.        , 0.94409966],
        [0.37070888, 0.30989102, 1.        , 1.        ],
        [0.44379634, 0.0839619 , 1.        , 1.        ],
        [0.36963248, 0.11252198, 1.        , 0.8905176 ],
        [0.4281221 , 0.        , 1.        , 0.7496372 ],
        [0.        , 0.        , 1.        , 0.82337487],
        [0.        , 0.11620069, 1.        , 1.        ],
        [0.23434713, 0.41739094, 1.        , 1.        ],
        [0.        , 0.        , 1.        , 1.        ],
        [0.        , 0.        , 1.        , 0.5409044 ],
        [0.        , 0.        , 1.        , 0.8676545 ],
        [0.1968269 , 0.21005818, 1.        , 1.        ],
        [0.        , 0.19769558, 0.8214438 , 1.        ],
        [0.        , 0.11620069, 1.        , 1.        ],

In [20]:
from matplotlib import pyplot as plt